###### picking the cell data points near to the GPS data location

In [1]:
import pandas as pd
import numpy as np
import os, glob
import datetime
import matplotlib.pyplot as plt

##### Merging cell and GPS data (nearest GPS location time)

In [2]:
outpath = "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/gps_test/"

##### cell data frames location

In [3]:
path = "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/entire_data/test/"
cell_files = glob.glob(os.path.join(path, "cell-*.csv"))
print(len(cell_files))
cfls = len(cell_files)

15


##### cell and GPS data frames reading

In [4]:
for cname in cell_files:
    cpart = cname.split("/")[-1]
    cpart1 = cpart[4:]
    cdf = pd.read_csv(cname, skiprows=1,low_memory=True, header=None)
    numcols = len(cdf.columns)
    print("fname: {0}; no. of cloumns: {1}".format(cname, numcols))
    cdf.rename(columns={0:"route", 1:"mode", 2:"cellinfo", 3:"cputime", 4:"dsptime", 5: "status", 6: "ncells",
                        7:"type", 8:"pci",9:"regi", 10:"rsrp", 11:"rsrq", 12:"sinr",13:"rssi",14:"level",
                        15:"csirsrp",16:"csirsrq",17:"csisinr",18:"cqisindx",19:"cqis"}, inplace = True)
    cdf.drop(["dsptime","csirsrp","csirsrq","csisinr","cqisindx","cqis"], axis=1, inplace=True)
    #skip the last row
    cdf = cdf[:-1]
    
    #change cputime to date-time format
    cdf['datetime'] = pd.to_datetime(cdf['cputime'], unit='ms',utc=True).map(lambda x: x.tz_convert('America/Los_Angeles'))
    cdf['date'] = pd.to_datetime(cdf['datetime']).dt.date
    cdf['time'] = pd.to_datetime(cdf['datetime']).dt.time
    cdf.drop(['datetime'],axis=1, inplace=True)
    
    cell_df = cdf
    cell_df.drop(['mode','status','regi','date','time'],axis=1, inplace=True)
    display(cell_df.head(3))
    
    #********************************************************************************************
    ###GPS dataframe reading corresponding cell data frame
    gname = path + 'gps' + cpart1
    gdf = pd.read_csv(gname, skiprows=1,low_memory=True, header=None)
    numcols = len(gdf.columns)
    print("fname: {0}; no. of cloumns: {1}".format(gname, numcols))
    
    gdf.rename(columns={0:"unk", 1:"unk", 2:"GPS", 3:"cputime", 4:"acc", 5: "Lat", 6: "Lon", 
                           7:"gpstime", 8:"speed",9:"bearing", 10:"altitude", 11:"provider"}, inplace = True)

    #skip the last row
    gdf = gdf[:-1]
    gdf = gdf.dropna()
    #change cputime to date-time format
    gdf['datetime'] = pd.to_datetime(gdf['cputime'], unit='ms',utc=True).map(lambda x: x.tz_convert('America/Los_Angeles'))
    gdf['date'] = pd.to_datetime(gdf['datetime']).dt.date
    gdf['time'] = pd.to_datetime(gdf['datetime']).dt.time
    gdf.drop(['datetime'],axis=1, inplace=True)
    
    gps_df = gdf
    gps_df.drop(['unk', 'unk','acc','gpstime','bearing','provider'],axis=1, inplace=True)
    display(gps_df.head(3))
    
    #******************************************************************
    #output file (after merging cell and gps dataframes)
    outfile = outpath + 'gps_cell_merge'+cpart1
    print(outfile)
    
    #*****************************************************************************************************
    #first calc. the time difference between cell and gps data and finding the nearest time to the GPS one
    newDataFrame = pd.DataFrame()
    npts = len(gps_df)
    for ii in range(npts):
        st_time = float(gps_df.cputime[ii])
        gps_newdf = gps_df.iloc[ii:ii+1]
        #display(gps_newdf) 
    
        cell_df['cputime'] = cell_df['cputime'].astype(float)
        cell_df['dcpu'] = st_time - cell_df['cputime']
        cell_df['dcpu'] = cell_df['dcpu'].abs()
        cell_newdf1 = cell_df[cell_df["dcpu"] == cell_df['dcpu'].min()]
        cell_newdf = cell_newdf1.iloc[0:1]
        #display(f"length of dcpu: {len(cell_newdf1)}")
        #display(cell_newdf)
        cell_newdf = cell_newdf.reset_index()
        gps_newdf = gps_newdf.reset_index()
        #display(cell_df)
        #display(f"rowno.:{ii},cell & GPS")
        res = pd.concat([gps_newdf, cell_newdf], axis=1)
        #display(res.head(2))
        #display('______________________________')
        newDataFrame = newDataFrame.append(res)
           
    newDataFrame.to_csv(outfile, index = False)  
    

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/entire_data/test/cell-null-unknown-2023-08-16-17-39-06-354000000-fill_out_your_user_id.csv; no. of cloumns: 20


/.pkg/qct/software/python/3.6.9/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


route cellinfo        cputime  ncells type    pci  rsrp  rsrq   sinr  \
0  ROUTE_D  CELL_SS  1692232746573     4.0  LTE  379.0 -80.0  -9.0 -200.0   
1  ROUTE_D  CELL_SS  1692232746608     4.0  LTE  379.0 -80.0  -9.0 -200.0   
2  ROUTE_D  CELL_SS  1692232746633     4.0  LTE  379.0 -80.0  -9.0 -200.0   

   rssi  level  
0 -51.0    4.0  
1 -51.0    4.0  
2 -51.0    4.0

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/entire_data/test/gps-null-unknown-2023-08-16-17-39-06-354000000-fill_out_your_user_id.csv; no. of cloumns: 12


GPS        cputime        Lat         Lon  speed   altitude        date  \
0  GPS  1692232749001  33.520418 -117.664533    0.0  32.524902  2023-08-16   
1  GPS  1692232749046  33.520418 -117.664536    0.0  32.524963  2023-08-16   
2  GPS  1692232750064  33.520416 -117.664542    0.0  32.676147  2023-08-16   

              time  
0  17:39:09.001000  
1  17:39:09.046000  
2  17:39:10.064000

...

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/entire_data/test/cell-null-unknown-2023-08-16-07-08-40-097000000-fill_out_your_user_id.csv; no. of cloumns: 20


route cellinfo        cputime  ncells type    pci  rsrp  rsrq   sinr  \
0  ROUTE_A  CELL_SS  1692194920284     4.0  LTE  379.0 -78.0  -8.0 -200.0   
1  ROUTE_A  CELL_SS  1692194920312     4.0  LTE  379.0 -78.0  -8.0 -200.0   
2  ROUTE_A  CELL_SS  1692194920337     4.0  LTE  379.0 -78.0  -8.0 -200.0   

   rssi  level  
0 -51.0    4.0  
1 -51.0    4.0  
2 -51.0    4.0

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/entire_data/test/gps-null-unknown-2023-08-16-07-08-40-097000000-fill_out_your_user_id.csv; no. of cloumns: 12


GPS        cputime        Lat         Lon  speed   altitude        date  \
0  GPS  1692194923350  33.520404 -117.664582   0.00  33.516052  2023-08-16   
1  GPS  1692194923387  33.520402 -117.664581   0.00  33.516052  2023-08-16   
2  GPS  1692194924399  33.520316 -117.664538   3.02  36.138672  2023-08-16   

              time  
0  07:08:43.350000  
1  07:08:43.387000  
2  07:08:44.399000

/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/gps_test/gps_cell_merge-null-unknown-2023-08-16-07-08-40-097000000-fill_out_your_user_id.csv
